#### Setup



In [ ]:
!pip install -q sentence-transformers
!pip install -q wikipedia-api
!pip install -q numpy
!pip install -q scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

### Load the Embedding Model:

In [1]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("Alibaba-NLP/gte-base-en-v1.5", trust_remote_code=True)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/72.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

configuration.py:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

### Fetch Text Content from Wikipedia:



In [3]:
!pip install wikipediaapi

ERROR: Could not find a version that satisfies the requirement wikipediaapi (from versions: none)
ERROR: No matching distribution found for wikipediaapi


In [4]:
# Install wikipediaapi before importing it in the same cell.
!pip install wikipediaapi

from wikipediaapi import Wikipedia
wiki = Wikipedia('RAGBot/0.0', 'en')
doc = wiki.page('Hayao_Miyazaki').text
paragraphs = doc.split('\n\n') # chunking


ERROR: Could not find a version that satisfies the requirement wikipediaapi (from versions: none)
ERROR: No matching distribution found for wikipediaapi


ModuleNotFoundError: No module named 'wikipediaapi'

In [ ]:
import textwrap


In [ ]:
for i, p in enumerate(paragraphs):
  wrapped_text = textwrap.fill(p, width=100)

  print("-----------------------------------------------------------------")
  print(wrapped_text)
  print("-----------------------------------------------------------------")


-----------------------------------------------------------------
Hayao Miyazaki (宮崎 駿 or 宮﨑 駿, Miyazaki Hayao, Japanese: [mijaꜜzaki hajao]; born January 5, 1941) is
a Japanese animator, filmmaker, and manga artist. A founder of Studio Ghibli, he has attained
international acclaim as a masterful storyteller and creator of Japanese animated feature films, and
is widely regarded as one of the most accomplished filmmakers in the history of animation. Born in
Tokyo City in the Empire of Japan, Miyazaki expressed interest in manga and animation from an early
age, and he joined Toei Animation in 1963. During his early years at Toei Animation he worked as an
in-between artist and later collaborated with director Isao Takahata. Notable films to which
Miyazaki contributed at Toei include Doggie March and Gulliver's Travels Beyond the Moon. He
provided key animation to other films at Toei, such as Puss in Boots and Animal Treasure Island,
before moving to A-Pro in 1971, where he co-directed Lupi

### Embed the Document:

In [ ]:
docs_embed = model.encode(paragraphs, normalize_embeddings=True)

In [ ]:
docs_embed.shape

(19, 768)

In [ ]:
docs_embed[0]

array([ 1.20695028e-02, -9.31202341e-03,  3.86870503e-02, -1.79368909e-02,
        1.90604851e-03, -2.30416246e-02,  4.13034782e-02,  4.27563265e-02,
        4.51744124e-02, -1.10855019e-02, -1.45723019e-03,  7.79734971e-03,
        2.79529858e-02,  2.05775425e-02, -1.38202095e-02,  1.35757541e-02,
        1.48336263e-02, -3.48629579e-02, -3.57357860e-02, -3.17235813e-02,
        1.70307525e-03,  7.33010052e-03, -2.75423471e-02,  4.06559929e-02,
       -7.05514662e-03, -3.68454196e-02, -4.07886207e-02, -1.76706165e-02,
       -2.05303356e-02, -7.42677925e-03,  2.92662717e-02, -1.18500590e-02,
        5.10038342e-03,  1.64353717e-02, -3.09135970e-02, -5.56215830e-02,
       -4.58039716e-02,  5.18231504e-02, -2.54022256e-02, -3.31459232e-02,
       -7.28017911e-02,  1.32487444e-02, -3.62599231e-02, -1.61632821e-02,
       -5.15350699e-02,  1.28653413e-02,  2.11366210e-02, -4.02928218e-02,
        3.80871515e-03, -2.21524592e-02, -3.45152691e-02,  1.30857376e-03,
       -9.98750795e-04, -

### Embed the Query:

In [ ]:
query = "What was Studio Ghibli's first film?"
query_embed = model.encode(query, normalize_embeddings=True)


In [ ]:
query_embed.shape

(768,)

### Find the Closest Paragraphs to the Query:



In [ ]:
import numpy as np
similarities = np.dot(docs_embed, query_embed.T)

In [ ]:
similarities.shape

(19,)

In [ ]:
similarities

array([0.5579109 , 0.46477073, 0.50707126, 0.5064434 , 0.43503475,
       0.5633704 , 0.68840396, 0.49852747, 0.5992924 , 0.48889732,
       0.45300704, 0.42018926, 0.45964807, 0.4028834 , 0.49752194,
       0.4371641 , 0.42018428, 0.13663508, 0.4650131 ], dtype=float32)

In [ ]:
top_3_idx = np.argsort(similarities, axis=0)[-3:][::-1].tolist()


In [ ]:
top_3_idx

[6, 8, 5]

In [ ]:
most_similar_documents = [paragraphs[idx] for idx in top_3_idx]

In [ ]:
CONTEXT = ""
for i, p in enumerate(most_similar_documents):
  wrapped_text = textwrap.fill(p, width=100)

  print("-----------------------------------------------------------------")
  print(wrapped_text)
  print("-----------------------------------------------------------------")
  CONTEXT += wrapped_text + "\n\n"

-----------------------------------------------------------------
Studio Ghibli Early films (1985–1996) On June 15, 1985, Miyazaki and Takahata founded the animation
production company Studio Ghibli as a subsidiary of Tokuma Shoten. Studio Ghibli's first film was
Laputa: Castle in the Sky (1986), directed by Miyazaki. Some of the architecture in the film was
also inspired by a Welsh mining town; Miyazaki witnessed the mining strike upon his first visit to
Wales in 1984 and admired the miners' dedication to their work and community. Laputa was released on
August 2, 1986, by the Toei Company. It sold around 775,000 tickets; Miyazaki and Suzuki expressed
their disappointment with the film's box office figures. Miyazaki's following film, My Neighbor
Totoro, was released alongside Takahata's Grave of the Fireflies in April 1988 to ensure Studio
Ghibli's financial status. My Neighbor Totoro features the theme of the relationship between the
environment and humanity, showing that harmony is t

In [ ]:
query = "What was Studio Ghibli's first film?"

In [ ]:
prompt = f"""
use the following CONTEXT to answer the QUESTION at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

CONTEXT: {CONTEXT}
QUESTION: {query}

"""

In [ ]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00


In [ ]:
# prompt: write python code to make calls to openai api
from google.colab import userdata
userdata.get('openai')

import openai



In [ ]:
from openai import OpenAI
client = OpenAI(api_key=userdata.get('openai'))

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": prompt},
  ]
)

In [ ]:
print(response.choices[0].message.content)


Studio Ghibli's first film was *Laputa: Castle in the Sky* (1986), directed by Hayao Miyazaki.
